In [2]:
import coremltools
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.lite.experimental.examples.lstm.rnn import bidirectional_dynamic_rnn
from tensorflow.keras.callbacks import LambdaCallback
import tensorflow.keras.backend as K

import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import string

import keras
from keras.models import Model,Sequential, Input, load_model
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [3]:
vocab=[]
entities=[]

with open('result_usbank_1.json') as json_file:
    dataset = json.load(json_file)
    for data in dataset:
    	sent=data['sentence']
    	tags=data['tags']
    	entities.extend(tags)
    	for word in sent:
    		vocab.append(word)
json_file.close()

vocab=set(vocab)
t=Tokenizer(num_words=len(vocab))
t.fit_on_texts(vocab)
tokens=t.word_index

num_words=len(tokens)+1

reverse_tokens = dict([(value, key) for key, value in tokens.items()])

entities=set(entities)
entities=sorted(entities)
entities=list(entities)
label_encoder=LabelEncoder()
int_encoded=label_encoder.fit_transform(entities)
label_cat_encoded=to_categorical(int_encoded,num_classes=len(entities))

master={}
for i in range(len(entities)):
	#master[entities[i]]=int_encoded[i]
	master[entities[i]]=i
print(master)

reverse_master = dict([(value, key) for key, value in master.items()])

{'amount': 0, 'bill_type': 1, 'other': 2, 'payment_day': 3, 'payment_month': 4, 'payment_type': 5}


In [4]:
def preparation(sent,tags,tokens,master):
    dummy_x=[]
    dummy_y=[]
    for word,tag in zip(sent,tags):
        dummy_x.append(tokens[word.lower()])
        dummy_y.append(master[tag])
    return dummy_x,dummy_y

def pre_process(sent,tokens):
    zero_tok=0.0
    sent=sent.split(" ")
    dummy_in=[]
    for word in sent:
        if word.lower() not in tokens:
            dummy_in.append(zero_tok)
        else:
            dummy_in.append(tokens[word.lower()])
    return dummy_in

In [5]:
max_length=20
X=[]
Y=[]
with open('result_usbank_1.json') as json_file:
    dataset = json.load(json_file)
    for data in dataset:
        sent=data['sentence']
        tags=data['tags']
        tokenized_x,encoded_y=preparation(sent,tags,tokens,master)
        X.append(tokenized_x)
        Y.append(encoded_y)

X=pad_sequences(X,maxlen=max_length,padding='post',dtype=float)
y=pad_sequences(Y,maxlen=max_length,padding='post')
y = y.reshape(y.shape[0],y.shape[1])
y = to_categorical(y,num_classes=len(master))
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.2)
len(X)

451

In [6]:
# # model_new = tf.keras.models.Sequential([
# #   tf.keras.Input(shape=(X.shape[1],), name='input'),
# #   tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=32,input_length=X.shape[1],trainable=True),
# #   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
# #   tf.keras.layers.Dense(len(entities),activation=tf.nn.softmax,name='output')
# # ])

input_layer=Input(shape=(X.shape[1],))
model=Embedding(input_dim=len(vocab)+1,output_dim=32,input_length=X.shape[1])(input_layer)
model = Bidirectional(LSTM(units = 32,return_sequences=True, recurrent_dropout=0.2)) (model)
output_layer= Dense(len(master), activation="softmax")(model)
model_new = Model(input_layer,output_layer)

model_new.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 32)            5408      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 64)            16640     
_________________________________________________________________
dense_1 (Dense)              (None, 20, 6)             390       
Total params: 22,438
Trainable params: 22,438
Non-trainable params: 0
_________________________________________________________________


In [7]:
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.2)

history = model_new.fit(train_x,train_y,epochs=100,batch_size=16)
model_new.save('custom_ner_usbank.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
360/360 [==============================] - 1s 2ms/step - loss: 1.6584 - accuracy: 0.5800
Epoch 2/100
360/360 [==============================] - 0s 508us/step - loss: 0.9663 - accuracy: 0.7433
Epoch 3/100
360/360 [==============================] - 0s 494us/step - loss: 0.6917 - accuracy: 0.8079
Epoch 4/100
360/360 [==============================] - 0s 584us/step - loss: 0.6171 - accuracy: 0.8263
Epoch 5/100
360/360 [==============================] - 0s 497us/step - loss: 0.5766 - accuracy: 0.8353
Epoch 6/100
360/360 [==============================] - 0s 493us/step - loss: 0.5469 - accuracy: 0.8379
Epoch 7/100
360/360 [==============================] - 0s 504us/step - loss: 0.5173 - accuracy: 0.8406
Epoch 8/100
360/360 [==============================] - 0s 465us/step - loss: 0.4812 - accuracy: 0.8422
Epoch 9/100
360/360 [==============================] - 0s 481us/step - loss: 0.4346 - accuracy: 0.8429
Epoch 10/100
360/360 [==============================] - 0s 548us/step - los

360/360 [==============================] - 0s 649us/step - loss: 0.0049 - accuracy: 0.9990
Epoch 80/100
360/360 [==============================] - 0s 827us/step - loss: 0.0050 - accuracy: 0.9990
Epoch 81/100
360/360 [==============================] - 0s 648us/step - loss: 0.0051 - accuracy: 0.9990
Epoch 82/100
360/360 [==============================] - 0s 573us/step - loss: 0.0048 - accuracy: 0.9990
Epoch 83/100
360/360 [==============================] - 0s 528us/step - loss: 0.0049 - accuracy: 0.9990
Epoch 84/100
360/360 [==============================] - 0s 512us/step - loss: 0.0049 - accuracy: 0.9990
Epoch 85/100
360/360 [==============================] - 0s 522us/step - loss: 0.0051 - accuracy: 0.9990
Epoch 86/100
360/360 [==============================] - 0s 568us/step - loss: 0.0047 - accuracy: 0.9990
Epoch 87/100
360/360 [==============================] - 0s 537us/step - loss: 0.0047 - accuracy: 0.9989
Epoch 88/100
360/360 [==============================] - 0s 570us/step - loss:

In [8]:
with open('vocab/vocab_ner_usbank.json','w') as f:
    json.dump(tokens,f)

In [11]:
temp='Pay the excel bill using my savings account tomorrow'
length=temp.split(" ")
tok_sent=pre_process(temp,tokens)
for i in range(max_length):
    if len(tok_sent)<max_length:
        tok_sent.append(0.0)
tok_sent=np.expand_dims(tok_sent,axis=0)
print(tok_sent)
result=model_new.predict(tok_sent)
for i in range(len(length)):
    ind=np.argmax(result[0][i])
    print(length[i],"----->",reverse_master[ind])

[[ 60. 116. 166.  12. 120.  87. 110.  82.  59.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
Pay -----> other
the -----> other
excel -----> bill_type
bill -----> other
using -----> other
my -----> other
savings -----> payment_type
account -----> other
tomorrow -----> payment_day


In [18]:
ner_label=['Date','Drug','Duration','Quantity','other']
ner_usbank=['amount','bill_type','other','payment_day','payment_month','payment_type']

coreml_model = coremltools.converters.keras.convert('custom_ner_usbank.h5',class_labels = ner_usbank)
coreml_model.save('models/custom_ner_usbank.mlmodel')

0 : input_1, <keras.engine.input_layer.InputLayer object at 0x1464559b0>
1 : embedding_1, <keras.layers.embeddings.Embedding object at 0x146455a20>
2 : bidirectional_1, <keras.layers.wrappers.Bidirectional object at 0x146432278>
3 : dense_1, <keras.layers.core.Dense object at 0x146455c50>
4 : dense_1__activation__, <keras.layers.core.Activation object at 0x14d90ca58>


In [19]:
import numpy as np
# tokens=[[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]]
# tokens=np.array(tokens,dtype=np.double)
# tokens = tokens.reshape(20,1,1)
token=[59,]
token=np.array(token,dtype=np.double)
token.shape

(1,)

In [20]:
coreml_model.predict({'input1':token})

{'bidirectional_1_c_out_rev': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'classLabel': 'payment_day',
 'bidirectional_1_h_out': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'bidirectional_1_c_out': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'bidirectional_1_h_out_rev': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'output1': {'amount': 0.06237495318055153,
  'bill_type': 0.05016864463686943,
  'other': 0.038017503917217255,
  'payment_day': 0.6281217336654663,
  'payment_month': 0.145807147026062,
  'payment_type': 0.07550998032093048}}